In [4]:
import pandas as pd
data_2021_2023 = pd.read_csv('Data/salaries.csv')
data_2021_2023.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Machine Learning Engineer,65000,EUR,70179,IT,100,US,L
1,2023,MI,FT,BI Developer,110000,USD,110000,US,0,US,M
2,2023,MI,FT,BI Developer,75000,USD,75000,US,0,US,M
3,2023,MI,FT,Data Analyst,43000,EUR,46426,SI,0,SI,M
4,2023,MI,FT,Data Analyst,36000,EUR,38868,SI,0,SI,M


In [7]:
data_2017_2020 = pd.read_csv('Data/2019_Data_Professional_Salary_Survey_Responses.csv')
data_2017_2020.head(len(data_2017_2020))

,Survey Year,Timestamp,SalaryUSD,Country,PostalCode,PrimaryDatabase,YearsWithThisDatabase,OtherDatabases,EmploymentStatus,JobTitle,...,HoursWorkedPerWeek,TelecommuteDaysPerWeek,PopulationOfLargestCityWithin20Miles,EmploymentSector,LookingForAnotherJob,CareerPlansThisYear,Gender,OtherJobDuties,KindsOfTasksPerformed,Counter
0,2017,1/5/2017 5:10:20,200000,United States,Not Asked,Microsoft SQL Server,10,MySQL/MariaDB,Full time employee,DBA,...,45,1,Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1
1,2017,1/5/2017 5:26:23,61515,United Kingdom,Not Asked,Microsoft SQL Server,15,"Oracle, PostgreSQL",Full time employee,DBA,...,35,2,Not Asked,Private business,No,Not Asked,Not Asked,Not Asked,Not Asked,1
2,2017,1/5/2017 5:32:57,95000,Germany,Not Asked,Microsoft SQL Server,5,"Oracle, MySQL/MariaDB, Informix",Full time employee,Other,...,45,"None, or less than 1 day per week",Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1
3,2017,1/5/2017 5:33:03,56000,United Kingdom,Not Asked,Microsoft SQL Server,6,NaN,Full time employee,DBA,...,40,1,Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1
4,2017,1/5/2017 5:34:33,35000,France,Not Asked,Microsoft SQL Server,10,Oracle,Full time employee of a consulting/contracting...,DBA,...,40,"None, or less than 1 day per week",Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6888,2019,1/4/2019 6:16:12,32000,Hungary,NaN,Microsoft SQL Server,3,"Azure SQL DB, Amazon RDS (any flavor)",Full time employee,DBA (General - splits time evenly between writ...,...,40,2,20K-99K (large town),Private business,No,"Stay with the same employer, same role",Male,NaN,"On-call as part of a rotation, Projects",1
6889,2019,1/4/2019 6:18:47,32000,Hungary,NaN,Microsoft SQL Server,3,"Azure SQL DB, Amazon RDS (any flavor)",Full time employee,DBA (General - splits time evenly between writ...,...,40,2,20K-99K (large town),Private business,No,"Stay with the same employer, same role",Male,NaN,Projects,1
6890,2019,1/4/2019 9:06:05,40000,Portugal,1000,Microsoft SQL Server,10,NaN,Full time employee of a consulting/contracting...,"Developer: App code (C#, JS, etc)",...,40,"None, or less than 1 day per week",300K-1M (large city),Private business,"Yes, but only passively (just curious)","Stay with the same employer, same role",Male,"DBA (Development Focus - tunes queries, indexe...","Manual tasks, Meetings & management, On-call a...",1
6891,2019,1/4/2019 10:06:24,130000,United States,49506,Microsoft SQL Server,20,NaN,Full time employee of a consulting/contracting...,DBA (General - splits time evenly between writ...,...,42,"None, or less than 1 day per week",100K-299K (city),Private business,"Yes, actively looking for something else",Prefer not to say,Female,NaN,"Build scripts & automation tools, Projects, R&D",1
